# Generate automated session reports for all participants

In [1]:
import pandas as pd
import numpy as np
import mne
import mne_bids
import glob
import warnings

import re
import subprocess

import os
import papermill as pm
warnings.filterwarnings('ignore')

from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor
from nbformat import read


### Select task parameters and file paths

In [2]:
## Select kind of data experimental or control group

#dataset_group = "experimental"

#if dataset_group == "control":
#    # Control group data
#    project_path = "/home/james/q1k/pilot/q1k-external-pilot/"
#    task_id_in = "ap"
#    task_id_in_et = "ap"
#    task_id_out = "ap"
#    #subject_id = '002'
#    session_id = '01'
#    run_id = '1'

#elif dataset_group == "experimental":
## Experimental group data

project_path = "/project/def-emayada/q1k/experimental/"
pyll_path = "derivatives/pylossless/"
sync_loss_path = "derivatives/sync_loss/"
segment_path = "derivatives/segment/"
code_path = "code/q1k_segment/"
out_seg_path = "epoch_fif_files/"
out_avg_path = "erp_fif_files/"
html_reports_path = "session_reports/"
task_id = "TO"
#task_id_in_et = "TO"
#task_id_out = "TO"
#subject_id = 'Q1K_HSJ_100123_F1'
run_id = '1'
session_id = '01'
#sourcedata_path = "sourcedata/" 
#html_reports_path = "session_reports/" + task_id_in  + '/'



In [3]:
# Sanity check to see which task you would like to make reports for 
print(task_id)

TO


In [4]:
# Function to extract details from filename
def extract_job_info(filename):
    # Define the pattern to capture the required sections
    pattern = r"sub-(.*?)_ses-(.*?)_task-(.*?)_run-(.*?)_eeg\.edf"
    match = re.match(pattern, filename)

    if match:
        # Extract the groups from the match
        subject_id = match.group(1)
        session_id = match.group(2)
        task_id = match.group(3)
        run_id = match.group(4)
        return subject_id, session_id, task_id, run_id
    else:
        raise ValueError("Filename pattern did not match.")


in_pattern = project_path + pyll_path + sync_loss_path + '**/eeg/*' + task_id + '*.edf'
in_file_paths = glob.glob(in_pattern, recursive=True)
print(f"Input files:")
for file_path in in_file_paths:
    file_name = os.path.basename(file_path)  # Extract the filename from the full path
    print(f"File Path: {file_path}, File Name: {file_name}")

    
# Make sure the output directories exists
if not os.path.exists(project_path + pyll_path + sync_loss_path + segment_path + out_seg_path + task_id):
    os.makedirs(project_path + pyll_path + sync_loss_path + segment_path + out_seg_path + task_id)
if not os.path.exists(project_path + pyll_path + sync_loss_path + segment_path + out_avg_path + task_id):
    os.makedirs(project_path + pyll_path + sync_loss_path +segment_path + out_avg_path + task_id)
    
out_seg_pattern = project_path + pyll_path + sync_loss_path + segment_path + out_seg_path + task_id + '/*' + task_id + '*epo.fif'
out_seg_file_paths = glob.glob(out_seg_pattern, recursive=True)
print(f"existing output epoch files:")
for file_path in out_seg_file_paths:
    file_name = os.path.basename(file_path)  # Extract the filename from the full path
    print(f"File Path: {file_path}, File Name: {file_name}")

out_avg_pattern = project_path + pyll_path + sync_loss_path + segment_path + out_avg_path + task_id + '/*' + task_id + '*epo.fif'
out_avg_file_paths = glob.glob(out_seg_pattern, recursive=True)
print(f"existing output avg files:")
for file_path in out_avg_file_paths:
    file_name = os.path.basename(file_path)  # Extract the filename from the full path
    print(f"File Path: {file_path}, File Name: {file_name}")

    #submit_slurm_job(project_path, file_name)

Input files:
File Path: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/sub-1102S1/ses-01/eeg/sub-1102S1_ses-01_task-TO_run-1_eeg.edf, File Name: sub-1102S1_ses-01_task-TO_run-1_eeg.edf
File Path: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/sub-1147F1/ses-01/eeg/sub-1147F1_ses-01_task-TO_run-1_eeg.edf, File Name: sub-1147F1_ses-01_task-TO_run-1_eeg.edf
File Path: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/sub-1033P/ses-01/eeg/sub-1033P_ses-01_task-TO_run-1_eeg.edf, File Name: sub-1033P_ses-01_task-TO_run-1_eeg.edf
File Path: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/sub-1067P/ses-01/eeg/sub-1067P_ses-01_task-TO_run-1_eeg.edf, File Name: sub-1067P_ses-01_task-TO_run-1_eeg.edf
File Path: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/sub-0157P/ses-01/eeg/sub-0157P_ses-01_task-TO_run-1_eeg.edf, File N

In [5]:
glob.glob(project_path + pyll_path + sync_loss_path + segment_path + code_path + html_reports_path + task_id + "/*.html")

[]

In [6]:
# Generate list of session reports already processed so they are not processed again in next step
processed_sessions = []

for subject in glob.glob(project_path + pyll_path + sync_loss_path + segment_path + code_path + html_reports_path + task_id + "/*.html"): 
    html_file = subject.split('/')[-1]
    processed_sessions.append(html_file.split('_')[0])

    #processed_sessions.append(subject.split('\\')[1][:-5])
    print(subject)
    #processed_sessions.append(subject.split('//')[1][:-5])

print('Existing session reports:')
processed_sessions

Existing session reports:


[]

## Generate html session reports for all participants

In [7]:
# Directory to save HTML reports
#html_reports_dir = './../../session_reports/' + task_id_in + '/'


# Create a list for participants with errors
error_subjects = []

#for file in glob.glob("./../../sourcedata/eeg/Q1K*/Q1K" + "*AEP" + '*.mff'):
#for file in glob.glob(project_path + sourcedata_path + "eeg/Q1K*/Q1K*" + task_id_in + '*.mff'):
for file in in_file_paths:
    
    file_name = os.path.basename(file)
    print('Current data file: ' + file_name)
    ## Select anything after the Q1K and before the AEP
    ##subject_id = file.split('_')[2]
    ##site_part = file.split("HSJ_")
    subject_id = file_name.split("_")[0].split("-")[1]
    print('Participant ID: ' + subject_id)

    #subject_relation = site_part[1].split("_")[1].split("/")[0]
    #print('Participant relation: ' + subject_relation)

    #subject_id_in = subject_number + "_" + subject_relation
    #print('Participant ID input: ' + subject_id_in)
    #subject_id_out = subject_number.replace('_','').replace('-','') + subject_relation
    #print('Participant ID output: ' + subject_id_out)
    
    # Skip sessions that have already been processed
    print(subject_id)
    #if subject_id in out_seg_file_paths:
    if subject_id in processed_sessions:
        print(subject_id + ' has already been processed')
        continue    

    # Define paths
    input_notebook = project_path + pyll_path + sync_loss_path + segment_path + code_path + 'session_seg_' + task_id + '.ipynb'

    ##output_notebook = f'./../../session_reports/{task_id_in}/executed_notebooks/{subject_id}_{task_id_in}_executed.ipynb'
    ##output_html = f'./../../session_reports/{task_id_in}/{subject_id}_{session_id}_{task_id_in}.html'
    ##output_html = f'{html_reports_dir}/{subject_id}.html'
    output_notebook = f'{project_path}{pyll_path}{sync_loss_path}{segment_path}{code_path}{html_reports_path}{task_id}/executed_notebooks/{subject_id}_{task_id}_executed.ipynb'
    print('Output notebook file: ' + output_notebook)
    output_html = f'{project_path}{pyll_path}{sync_loss_path}{segment_path}{code_path}{html_reports_path}{task_id}/{subject_id}_{task_id}.html'
    print('Output HTML file: ' + output_html)

    try:
        # Execute the notebook
        pm.execute_notebook(input_notebook, output_notebook, kernel_name = 'q1k_env', parameters=dict(subject_id=subject_id, task_id=task_id,
        session_id=session_id, run_id=run_id))

        # Convert executed notebook to HTML
        html_exporter = HTMLExporter()
        html_exporter.exclude_input = True

        (body, resources) = html_exporter.from_filename(output_notebook)

        # Save HTML output
        with open(output_html, 'w', encoding='utf-8') as f:
            f.write(body)

        print(f"HTML report saved for {subject_id}.")
    

    except Exception as e:
        # Handle the error 
        error_subjects.append(subject_id)
        print(f"Error while processing {subject_id}: {e}")

        # Print out the list of subjects with errors
print( "These subjects have errors: " + str(error_subjects) + " and need to be reprocessed")

Current data file: sub-1102S1_ses-01_task-TO_run-1_eeg.edf
Participant ID: 1102S1
1102S1
Output notebook file: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/derivatives/segment/code/q1k_segment/session_reports/TO/executed_notebooks/1102S1_TO_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/derivatives/segment/code/q1k_segment/session_reports/TO/1102S1_TO.html


Executing:   0%|          | 0/31 [00:00<?, ?cell/s]

HTML report saved for 1102S1.
Current data file: sub-1147F1_ses-01_task-TO_run-1_eeg.edf
Participant ID: 1147F1
1147F1
Output notebook file: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/derivatives/segment/code/q1k_segment/session_reports/TO/executed_notebooks/1147F1_TO_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/derivatives/segment/code/q1k_segment/session_reports/TO/1147F1_TO.html


Executing:   0%|          | 0/31 [00:00<?, ?cell/s]

HTML report saved for 1147F1.
Current data file: sub-1033P_ses-01_task-TO_run-1_eeg.edf
Participant ID: 1033P
1033P
Output notebook file: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/derivatives/segment/code/q1k_segment/session_reports/TO/executed_notebooks/1033P_TO_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/derivatives/segment/code/q1k_segment/session_reports/TO/1033P_TO.html


...

HTML report saved for 1073P.
Current data file: sub-1143F1_ses-01_task-TO_run-1_eeg.edf
Participant ID: 1143F1
1143F1
Output notebook file: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/derivatives/segment/code/q1k_segment/session_reports/TO/executed_notebooks/1143F1_TO_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/derivatives/segment/code/q1k_segment/session_reports/TO/1143F1_TO.html


Executing:   0%|          | 0/31 [00:00<?, ?cell/s]

HTML report saved for 1143F1.
Current data file: sub-0154P_ses-01_task-TO_run-1_eeg.edf
Participant ID: 0154P
0154P
Output notebook file: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/derivatives/segment/code/q1k_segment/session_reports/TO/executed_notebooks/0154P_TO_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/derivatives/pylossless/derivatives/sync_loss/derivatives/segment/code/q1k_segment/session_reports/TO/0154P_TO.html


Executing:   0%|          | 0/31 [00:00<?, ?cell/s]

HTML report saved for 0154P.
These subjects have errors: ['0171P', '0196M1'] and need to be reprocessed
